# §2-Extra カメラの投影変換

この章では投影変換の実装方法を学びましょう。

§2 で投影変換 (=ワールド座標系から画像座標系への変換) の大まかな流れを学びました。
ここでは Open3D・numpy を使った投影変換の実装方法を確認しましょう。

1. はじめに、ピンホールカメラでの投影変換の実装方法を確認しましょう。
2. また、最後により複雑なカメラモデルである魚眼レンズカメラモデルにも触れてみましょう。


In [1]:
from __future__ import annotations

import numpy as np
import open3d as o3d
import sympy as sp
from IPython.display import Math, display
from scipy.spatial.transform import Rotation

from util_lib.camera import (
    OCamCalibOmniDirectionalCamera,
    OCamCalibOmniDirectionalCameraParameters,
    PinholeCameraParameters,
    SimplePinholeCamera,
)
from util_lib.transformable_object import TransformableObject
from util_lib.types import EulerOrder, ICamera, Transform
from util_lib.visualization import draw_geometries

# ベースとなるカメラ
pinhole_camera_intrinsic = PinholeCameraParameters(focal_length=600,
                                                   principal_point=(1200 / 2 - 0.5, 900 / 2 - 0.5),
                                                   image_size=(1200, 900))
initial_rotate = Rotation.from_euler(EulerOrder.xyz, [0, 0, 0], degrees=True).as_matrix()
initial_pose = Transform.from_rotate_and_translate(initial_rotate, [0, 0, 0])
view_camera = SimplePinholeCamera(pinhole_camera_intrinsic, initial_pose)


# 行列表示用の関数
def print_matrix(mat: np.ndarray | Transform) -> None:
    if isinstance(mat, Transform):
        mat_sym = sp.Matrix(mat.get_matrix())
    else:
        mat_sym = sp.Matrix(mat)
    mat_sym = mat_sym.applyfunc(lambda x: sp.Symbol(f"{x:.3f}"))
    display(Math(sp.latex(mat_sym)))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## ワールド

例として、下図のようなグリッド状の点群が設置されたワールドを考えてみましょう。

点群の色は (赤, 緑, 青) の組で表され、ワールド座標の +x, +y, +z 方向にそれぞれ赤、緑、青の値が大きくなります。

<img alt="World settings" src="./notebook_assets/images/2_extra_world_settings.png" style="width:600px;" />

In [2]:
def generate_grid_cube_point_cloud(
        num_points_per_axis: int = 50, cube_size: float=10, epsilon: float=0.0,
    ) -> o3d.geometry.PointCloud:
    """Generate a grid cube with the specified number of points per axis and cube size."""
    line = np.linspace(0, num_points_per_axis, num_points_per_axis+1, dtype=np.float64)
    scale = cube_size / num_points_per_axis

    points = []
    colors = []
    rng = np.random.default_rng()
    for x in line:
        for y in line:
            for z in line:
                # 点が正確にグリッド上にあるとモアレが発生して見辛いので、座標値に適度な揺らぎを与える
                points.append(
                    [rng.normal(x, epsilon)*scale, rng.normal(y, epsilon)*scale, rng.normal(z, epsilon)*scale])
                colors.append([x/num_points_per_axis, y/num_points_per_axis, z/num_points_per_axis])

    # Open3D の点群オブジェクトの作り方
    # 1. 空の PointCloud インスタンスを生成
    # 2. 点の座標と色をそれぞれ、points 属性と colors 属性に設定する
    #    - points と colors は点の数を N とすると、N x 3 の配列である
    #    - points と colors は o3d.utility.Vector3dVector に変換したうえで設定する
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd


In [3]:
# カメラオブジェクト
base_camera_obj = TransformableObject.load_model("data/camera.gltf")

# カメラ (初期姿勢をカメラオブジェクトに合わせる)
pinhole_camera_intrinsic = PinholeCameraParameters(focal_length=600,
                                                   principal_point=(1200 / 2 - 0.5, 900 / 2 - 0.5),
                                                   image_size=(1200, 900))
initial_rotate = Rotation.from_euler(EulerOrder.xyz, [-90, 180, 0], degrees=True).as_matrix()
initial_pose = Transform.from_rotate_and_translate(initial_rotate, [0, 0, 0])
base_camera = SimplePinholeCamera(pinhole_camera_intrinsic, initial_pose)

# ワールド座標軸
world_coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])

# 画像座標軸 (後で使用)
image_coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=500, origin=[0, 0, 0])

# 立方体状の点群
cube = generate_grid_cube_point_cloud(20, 5, 0.05)

In [ ]:
draw_geometries(world_coordinate, cube, base_camera_obj, title="ワールドセッティング")

## ピンホールカメラによる投影変換

### カメラのセッティング

最初に、カメラを点群の正面に移動しましょう。そして Open3D のカメラビューでの見え方を確認しましょう。

下の左図はカメラと点群の位置関係を、右図はカメラに写る景色を示しています。

<img alt="Camera settings" src="./notebook_assets/images/2_extra_camera_in_front_of_cube.png" style="width:600px;" />

<img alt="Open3D camera view" src="./notebook_assets/images/2_extra_camera_view_in_front_of_cube.png" style="width:600px;" />

In [5]:
cube_center = np.mean(np.asarray(cube.points), axis=0)

camera_obj = base_camera_obj.copy()
# cube の中心を見るようにカメラを移動、またカメラを少し後ろに引く
camera_obj.translate([cube_center[0], -3, cube_center[2]])

camera = base_camera.copy()
camera.transform(camera_obj.get_transform())

draw_geometries(world_coordinate, cube, camera_obj, camera=camera, title="カメラセッティング")

### ピンホールカメラモデルによる投影変換の実装方法

§2 で学んだように、下記の手順で投影変換を行います。

1. 事前準備として、変換対象の点のワールド座標のセットを得ます
   * 点の数が $N$ であるとき、点の座標のセットを $3 \times N$ の行列にまとめます
   * Transformation のため、要素が全て `1` である行を追加し、$4 \times N$ 行列にします
   * また、ここでは点の座標を Open3D の PointCloud オブジェクトから抽出します
2. 点のワールド座標にカメラ外部パラメータ行列をかけて、カメラ座標に変換します
3. カメラ座標から正規画像座標に変換します
   * カメラ座標の X, Y 座標を Z 座標で割り、カメラ正面の距離1単位にある仮想的なスクリーン上の座標にします
4. 点の正規画像座標にカメラ行列をかけて、画像座標に変換します
5. 追加でカメラに写っている点だけをフィルターします
   1. ピンホールカメラではカメラの前方の点しか写らないので。カメラ座標への変換後にカメラ前方 (Z > 0) の点だけ残します
   2. 画像座標への変換後、画像内部 (0 <= X <= 画像の幅 ・ 0 <= Y <= 画像の高さ) にある点だけ残します

後の `projection_by_pinhole_camera()` 関数は上記の手順に従ってピンホールカメラモデルでの投影変換を実装したものです。この関数では、投影変換後の点の座標を使って新しい PointCloud オブジェクトを生成して返しています。


下の左図は、点群が正規画像座標に変換された様子を示しています。図中の座標軸はカメラ座標系の座標軸です。
点群が平面上になっていて、その中央が Z 軸の先端に接していることがわかります。

下の右図は、点群が画像座標に変換された様子を示しています。図中の座標軸は画像座標系の座標軸で、奥方向が +Z すなわちカメラの視線方向を、+X が画像の右方向を、+Y が画像の下方向を向いています。
点群が平面上になっていて、Open3D のカメラビューでの見え方と同じようになっていることがわかります。

<img alt="Projected cube" src="./notebook_assets/images/2_extra_cube_on_the_screen.png" style="width:600px;" />
<img alt="Projected cube" src="./notebook_assets/images/2_extra_projected_cube_from_front.png" style="width:600px;" />


カメラモデルによらず、ワールド座標からカメラ座標への変換は外部パラメータ行列を用いて統一的に行うことができます。
その一方で、カメラ座標から画像座標への変換は、カメラモデルに依存して定義が異なります。

このチュートリアルで定義した `ICamera` インターフェースでは、ワールド座標から画像座標への一般化された変換を `world_to_camera()` メソッドで定義しています。

後の `projection_by_camera()` 関数は、`ICamera` を実装したクラスの `world_to_camera()` メソッドを利用して投影変換を行います。

In [6]:
def projection_by_pinhole_camera(
        pcd: o3d.geometry.PointCloud,
        camera: SimplePinholeCamera,
        *,
        return_with_color: bool = True) -> tuple[o3d.geometry.PointCloud, o3d.geometry.PointCloud]:
    """
    カメラの外部パラメータと内部パラメータを用いてステップバイステップで投影変換を行う.

    Note:
    ----
    ピンホールカメラのように投影変換が行列の積で表現できることを前提とする

    """

    def filter_visible_points(points: np.ndarray, image_width: int, image_height: int) -> np.ndarray:
        # Filter points that are inside the image
        mask = np.where(
            (points[0, :] >= 0) & (points[0, :] <= image_width) & # 0 <= x <= image_width [pixel]
            (points[1, :] >= 0) & (points[1, :] <= image_height)) # 0 <= y <= image_height [pixel]
        return points[:, *mask], mask

    extrinsic_matrix = camera.get_extrinsic_matrix()
    camera_matrix = camera.get_intrinsic_matrix()
    image_size = camera.get_image_size()

    print("Extrinsic matrix (4x4)")  # noqa: T201 説明用にあえて print を使用
    print_matrix(extrinsic_matrix)
    print("Camera matrix (3x3)") # noqa: T201 説明用にあえて print を使用
    print_matrix(camera_matrix)

    # Open3D の点群の座標は Nx3 行列になっている
    # 外部パラメータ行列をかけるため、転置し、さらに全てが1の行をを追加して4xN 行列にする
    points = np.asarray(pcd.points).T
    points = np.vstack([points, np.ones(points.shape[1])])
    print(f"Input points ({points.shape[0]}x{points.shape[1]}) (first 5 points)") # noqa: T201 説明用にあえて print を使用
    print_matrix(points[:, :(min(5, points.shape[1]))])


    # ワールド座標系からカメラ座標系への変換
    points_in_camera = extrinsic_matrix @ points
    # カメラの前方にある点のみを残す
    mask_in_front_of_camera = points_in_camera[2, :] > 0
    points_in_camera = points_in_camera[:, mask_in_front_of_camera]

    # カメラ座標系の点を正規化 (正規画像座標系への変換)
    points_in_camera[:3] = points_in_camera[:3] / points_in_camera[2]

    # カメラ座標系 (正規画像座標系) から画像座標系への変換
    points_in_image = camera_matrix @ points_in_camera[:3]
    # スクリーンの点群の座標は 0 <= x <= image_width, 0 <= y <= image_height であるため、範囲外の点を除去
    points_in_image, mask_inside_image = filter_visible_points(points_in_image, image_size[0], image_size[1])


    pcd_in_camera = o3d.geometry.PointCloud()
    pcd_in_camera.points = o3d.utility.Vector3dVector(points_in_camera[:3].T)

    projected_pcd = o3d.geometry.PointCloud()
    projected_pcd.points = o3d.utility.Vector3dVector(points_in_image.T)

    if return_with_color:
        colors = np.asarray(pcd.colors)
        colors = colors[mask_in_front_of_camera]
        pcd_in_camera.colors = o3d.utility.Vector3dVector(colors)
        projected_pcd.colors = o3d.utility.Vector3dVector(colors[mask_inside_image])

    return projected_pcd, pcd_in_camera


def projection_by_camera(
        pcd: o3d.geometry.PointCloud,
        camera: ICamera,
        *,
        return_with_color: bool = True) -> o3d.geometry.PointCloud:
    """
    ICamera の world_to_camera メソッドを使用して点群を投影するバージョン.

    Note:
    ----
    カメラモデルによってはカメラ座標から画像座標への変換が単純な行列の積で表せない場合がある。
    ICamera ではワールド座標から画像座標への一般化された変換を world_to_camera で定義する。

    """
    points = np.asarray(pcd.points).T
    projected_points, mask_in_front_of_camera, mask_inside_image = camera.world_to_camera(points)

    projected_pcd = o3d.geometry.PointCloud()
    projected_pcd.points = o3d.utility.Vector3dVector(projected_points.T)
    if return_with_color:
        projected_pcd.colors = o3d.utility.Vector3dVector(
            np.asarray(pcd.colors)[mask_in_front_of_camera][mask_inside_image])
    return projected_pcd

In [7]:
cube_center = np.mean(np.asarray(cube.points), axis=0)

camera_obj = base_camera_obj.copy()
# cube の中心を見るようにカメラを移動、またカメラを少し後ろに引く
camera_obj.translate([cube_center[0], -3, cube_center[2]])

camera = base_camera.copy()
camera.transform(camera_obj.get_transform())

projected, normalized = projection_by_pinhole_camera(cube, camera, return_with_color=True)

draw_geometries(normalized, world_coordinate, camera=view_camera,
                title="正規画像座標系に変換された点群 (座標軸はカメラ座標系)")
draw_geometries(projected, image_coordinate, title="投影変換された点群 (座標軸は画像座標系)")

Extrinsic matrix (4x4)


<IPython.core.display.Math object>

Camera matrix (3x3)


<IPython.core.display.Math object>

Input points (4x9261) (first 5 points)


<IPython.core.display.Math object>

#### ピンホールカメラによる投影変換 (別の例)

今度は、カメラの位置を点群の角に移動し、カメラの向きを立方体の中心に向けてみましょう。

カメラの位置を (x, y, z) = (-1, -1, -1) に移動し、さらに Euler 角 (45°, 45°, 0) (回転順序 x → y → z) で回転させます。

下の左図はカメラと点群の位置関係を、右図は投影変換の結果を示しています。

<img alt="Camera at a corner of the cube" src="./notebook_assets/images/2_extra_camera_at_corner_of_cube.png" style="width:600px;" />
<img alt="Projected cube" src="./notebook_assets/images/2_extra_projected_cube_from_corner.png" style="width:600px;" />

In [8]:
cube_center = np.mean(np.asarray(cube.points), axis=0)

camera_obj = base_camera_obj.copy()
camera_obj.translate([-1, -1, -1])
camera_obj.rotate_by_euler(EulerOrder.xyz, [45, 45, 0])

camera = base_camera.copy()
camera.transform(camera_obj.get_transform())

draw_geometries(world_coordinate, cube, camera_obj, camera=camera, title="カメラセッティング")

projected = projection_by_camera(cube, camera)

draw_geometries(
    projected, o3d.geometry.TriangleMesh.create_coordinate_frame(size=500, origin=[0, 0, 0]), title="投影変換の結果")

## (参考) 魚眼レンズカメラのシミュレーション

ここでは応用例として、魚眼レンズモデルによる投影変換を紹介します。

魚眼カメラのモデルとして、[OCam Calib](https://sites.google.com/site/scarabotix/ocamcalib-omnidirectional-camera-calibration-toolbox-for-matlab) で定義されている Omnidirectional camera model を使用します。

OCam Calib では、ワールド座標から正規画像座標への変換を、光軸からの距離を変数とする多項式で表現します。

具体的な実装は [`util_lib/camera.py`](./util_lib/camera.py) の `OCamCalibFishEyeCamera`


<img alt="Camera in the cube" src="./notebook_assets/images/2_extra_camera_in_cube.png" style="height:600px;" />
<img alt="Projected cube by the fish eye camera" src="./notebook_assets/images/2_extra_projected_cube_from_inside_by_fish_eye.png" style="height:600px;" />

In [8]:
# Fish eye lense example: [Sunex DSL415](https://www.optics-online.com/OOL/DSL/DSL415.PDF)
# Camera system example : [NavVis VLX2](https://www.navvis.com/vlx-2)

fish_eye_param = OCamCalibOmniDirectionalCameraParameters(
    world_to_cam_params=[
        2175.0684380805101,
        1363.2716547833479,
        -221.43701011679701,
        -328.90812452717728,
        161.774047285835,
        629.14819508054643,
        -124.82718560288779,
        -981.25167223042808,
        -11.46872731590156,
        1240.455151263159,
        395.67008688719937,
        -1010.9145248169081,
        -668.30097100604326,
        382.03149432595819,
        479.82026899885392,
        41.138604370760731,
        -120.6327422172762,
        -56.594318327261128,
        -8.0040880097076297,
    ],
    affine_params_cde=(
        1.000233130570753,
        -0.00047428370003433208,
        0.00057600897530624498,
    ),
    principal_point=(5000 / 2 - 0.5, 5000 / 2 - 0.5),
    image_size=(5000, 5000),
    fov=195,
)

base_fish_eye_camera = OCamCalibOmniDirectionalCamera(fish_eye_param, initial_pose)

In [9]:
cube_center = np.mean(np.asarray(cube.points), axis=0)

camera_obj = base_camera_obj.copy()
# cube の中心を見るようにカメラを移動、またカメラを少し後ろに引く
camera_obj.translate(cube_center * 0.75)
camera_obj.rotate_by_euler(EulerOrder.xyz, [45, 45, 0])

fish_eye_camera = base_fish_eye_camera.copy()
fish_eye_camera.transform(camera_obj.get_transform())

projected = projection_by_camera(cube, fish_eye_camera)

draw_geometries(world_coordinate, cube, camera_obj, title="カメラセッティング")
draw_geometries(projected, o3d.geometry.TriangleMesh.create_coordinate_frame(size=2000, origin=[0, 0, 0]),
                title="魚眼レンズカメラモデルでの投影変換の結果")

下図はピンホールカメラによる投影変換の結果を示しています。

魚眼レンズカメラのほうが広範囲が写り込んでおり、そのために元の空間の構造が変形して見えることがわかるでしょう。

<img alt="Projected by pin-hole camera inside the cube" src="./notebook_assets/images/2_extra_projected_cube_from_inside_by_pinhole.png" style="width:600px;" />

In [10]:
pinhole_camera = base_camera.copy()
pinhole_camera.transform(camera_obj.get_transform())

projected_pinhole = projection_by_camera(cube, pinhole_camera)
draw_geometries(projected_pinhole, image_coordinate, title="ピンホールカメラモデルでの投影変換の結果")